## 1. 改变对象的字符串显示

In [1]:
# 要改变一个实例的字符串表示，可重新定义他的__str__()和__repr__()方法。
class Pair:
    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return 'Pair({0.x!r},{0.y!r})'.format(self)
    
    def __str__(self):
        return '({0.x!s},{0.y!s})'.format(self)

`__repr__()` 方法返回一个实例的代码表示形式，通常用来重新构造这个实例。 内置的 repr() 函数返回这个字符串，跟我们使用交互式解释器显示的值是一样的。` __str__()` 方法将实例转换为一个字符串，使用 str() 或 print() 函数会输出这个字符串。比如：

In [2]:
p = Pair(3,4) # __repr__() output
p

Pair(3,4)

In [3]:
print(p) # __str__() output

(3,4)


我们在这里还演示了在格式化的时候怎样使用不同的字符串表现形式。 特别来讲，!r 格式化代码指明输出使用 `__repr__() `来代替默认的 `__str__() `。 你可以用前面的类来试着测试下：

In [4]:
p = Pair(3,4)

In [5]:
print('p is {0!r}'.format(p))

p is Pair(3,4)


In [6]:
print('p is {0}'.format(p))

p is (3,4)


>> 自定义 `__repr__()` 和 `__str__() `通常是很好的习惯，因为它能简化调试和实例输出。 例如，如果仅仅只是打印输出或日志输出某个实例，那么程序员会看到实例更加详细与有用的信息。

## 2. 自定义字符串的格式化

In [7]:
# 为了自定义字符串的格式化，我们需要在类上面定义__format__()方法。
_format = {
    'ymd':'{d.year}-{d.month}-{d.day}',
    'mdy':'{d.month}/{d.day}/{d.year}',
    'dmy':'{d.day}/{d.month}/{d.year}'
}

class Date:
    def __init__(self,year,month,day):
        self.year = year
        self.month = month
        self.day = day
        
    def __format__(self,code):
        if code == '':
            code = 'ymd'
        fmt = _format[code]
        return fmt.format(d=self)

In [8]:
# 现在Date类的实例可以支持格式化操作了，如同下面这样：
d = Date(2012,12,21)

In [9]:
format(d)

'2012-12-21'

In [10]:
format(d,'mdy')

'12/21/2012'

In [11]:
'The date is {:ymd}'.format(d)

'The date is 2012-12-21'

In [12]:
'The date is {:mdy}'.format(d)

'The date is 12/21/2012'

`__format__() `方法给Python的字符串格式化功能提供了一个钩子。 这里需要着重强调的是格式化代码的解析工作完全由类自己决定。因此，格式化代码可以是任何值。 例如，参考下面来自 datetime 模块中的代码：

In [13]:
from datetime import date

In [14]:
d = date(2012,12,21)

In [15]:
format(d)

'2012-12-21'

In [16]:
format(d,'%A, %B %d, %Y')

'Friday, December 21, 2012'

In [17]:
'The end is {:%d %b %Y}. Goodbye'.format(d)

'The end is 21 Dec 2012. Goodbye'

## 3. 让对象支持上下文管理协议

In [18]:
# 为了让一个对象兼容with语句，你需要实现__enter__()和__exit__()方法。
from socket import socket,AF_INET,SOCK_STREAM

class LazyConnection:
    def __init__(self,address,family=AF_INET,type=SOCK_STREAM):
        self.address = address
        self.family = family
        self.type = type
        self.sock = None
        
        
    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('Already connected')
        self.sock = socket(self.family,self.type)
        self.sock.connect(self.address)
        return self.sock
    
    def __exit__(self,exc_ty,exc_val,tb):
        self.sock.close()
        self.sock = None

这个类的关键特点在于它表示了一个网络连接，但是初始化的时候并不会做任何事情(比如它并没有建立一个连接)。 连接的建立和关闭是使用 with 语句自动完成的，例如：

In [19]:
from functools import partial

conn = LazyConnection(('www.python.org',80))

# connection closed
with conn as s:
    # conn.__enter__() executes: connection open
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host:www.python.org\r\n')
    s.send(b'\r\n')
    resp = b''.join(iter(partial(s.recv,8192),b''))
    # conn.__exit__() executes: connection closed

编写上下文管理器的主要原理是你的代码会放到 with 语句块中执行。 当出现 with 语句的时候，对象的 `__enter__()` 方法被触发， 它返回的值(如果有的话)会被赋值给 as 声明的变量。然后，with 语句块里面的代码开始执行。 最后，`__exit__() `方法被触发进行清理工作。

不管 with 代码块中发生什么，上面的控制流都会执行完，就算代码块中发生了异常也是一样的。 事实上，`__exit__() `方法的第三个参数包含了异常类型、异常值和追溯信息(如果有的话)。 `__exit__()` 方法能自己决定怎样利用这个异常信息，或者忽略它并返回一个None值。 如果 `__exit__()` 返回 True ，那么异常会被清空，就好像什么都没发生一样， with 语句后面的程序继续在正常执行。



## 4. 创建大量对象时节省内存方法

对于主要是用来当成简单的数据结构的类而言，你可以通过给类添加__slots__()属性来极大的减少实例所占的内存。比如：

In [20]:
class Date:
    __slots__ = ['year','month','day']
    
    def __init__(self,year,month,day):
        self.year = year
        self.month = month
        self.day = day

当你定义 `__slots__ `后，Python就会为实例使用一种更加紧凑的内部表示。 实例通过一个很小的固定大小的数组来构建，而不是为每个实例定义一个字典，这跟元组或列表很类似。 在 `__slots__` 中列出的属性名在内部被映射到这个数组的指定小标上。 使用slots一个不好的地方就是我们不能再给实例添加新的属性了，只能使用在` __slots__ `中定义的那些属性名。

>> 尽管slots看上去是一个很有用的特性，很多时候你还是得减少对它的使用冲动。 Python的很多特性都依赖于普通的基于字典的实现。 另外，定义了slots后的类不再支持一些普通类特性了，比如多继承。 大多数情况下，你应该只在那些经常被使用到的用作数据结构的类上定义slots (比如在程序中需要创建某个类的几百万个实例对象)。

## 5. 在类中封装属性名

Python程序员不去依赖语言特性去封装数据，而是通过遵循一定的属性和放啊放命名规约和方法命名规约来达到这个效果。第一个约定是任何以单下划线_开头的名字都应该是内部实现。

In [21]:
class A:
    
    def __init__(self):
        self.internal = 0 # An internal attribute
        self.public = a # A public attribute
        
    def public_method(self):
        '''
        A public method
        '''
        pass
    
    def _internal_method(self):
        pass

Python并不会真的阻止别人访问内部名称。但是如果你这么做肯定是不好的，可能会导致脆弱的代码。 同时还要注意到，使用下划线开头的约定同样适用于模块名和模块级别函数。 例如，如果你看到某个模块名以单下划线开头(比如_socket)，那它就是内部实现。 类似的，模块级别函数比如 sys._getframe() 在使用的时候就得加倍小心了。

In [22]:
# 你还可能会遇到在类中定义中使用两个下划线（_）开头的命名。
class B:
    
    def __init__(self):
        self.private = 0
        
    def __private_method(self):
        pass
    
    def public_method(self):
        pass
        self.__private_method()

使用双下划线开始会导致访问名称变成其他形式。 比如，在前面的类B中，私有属性会被分别重命名为 `_B__private` 和` _B__private_method` 。 这时候你可能会问这样重命名的目的是什么，答案就是继承——这种属性通过继承是无法被覆盖的。比如：

In [23]:
class C(B):
    
    def __init__(self):
        super().__init__()
        self.__private = 1 # Does not override B._private
        
    # does not override B.__private_method()
    def __private_method(self):
        pass

>> 这里，私有名称` __private `和` __private_method `被重命名为 `_C__private `和` _C__private_method `，这个跟父类B中的名称是完全不同的。

上面提到有两种不同的编码约定(单下划线和双下划线)来命名私有属性，那么问题就来了：到底哪种方式好呢？ 大多数而言，你应该让你的非公共名称以单下划线开头。但是，如果你清楚你的代码会涉及到子类， 并且有些内部属性应该在子类中隐藏起来，那么才考虑使用双下划线方案。

## 6. 创建可管理的属性

如果你想给某个实例attribute增加除访问与修改之外的其他处理逻辑，比如类型检查或合法性验证。解决办法是：自定义某个属性的一种简单方法是将它定义为一个property。 例如，下面的代码定义了一个property，增加对一个属性简单的类型检查：

In [24]:
class Person:
    
    def __init__(self,first_name):
        self.first_name = first_name
        
    # Getter function
    @property
    def first_name(self):
        return self._first_name
    
    # Setter function
    @first_name.setter
    def first_name(self,value):
        if not isinstance(value,str):
            raise TypeError("Expected a string")
        self._first_name = value
        
    # Deleter function (optional)
    @first_name.deleter
    def first_name(self):
        raise AttributeError("Can't delete attribute")

>> 上述代码中有三个相关联的方法，这三个方法的名字都必须一样。 第一个方法是一个 **getter** 函数，它使得 **first_name** 成为一个属性。 其他两个方法给 **first_name** 属性添加了 **setter** 和 **deleter** 函数。 需要强调的是只有在 **first_name** 属性被创建后， 后面的两个装饰器 **@first_name.setter** 和 **@first_name.deleter** 才能被定义。

property的一个关键特征是它看上去跟普通的attribute没什么两样， 但是访问它的时候会自动触发 getter 、setter 和 deleter 方法。例如：

In [25]:
a = Person('Guido')

In [26]:
a.first_name

'Guido'

In [27]:
# Calls the setter
a.first_name = 42

TypeError: Expected a string

In [ ]:
del a.first_name

在实现一个property的时候，底层数据(如果有的话)仍然需要存储在某个地方。 因此，在get和set方法中，你会看到对 _first_name 属性的操作，这也是实际数据保存的地方。 另外，你可能还会问为什么 `__init__() `方法中设置了 `self.first_name` 而不是 `self._first_name `。 在这个例子中，我们创建一个property的目的就是在设置attribute的时候进行检查。 因此，你可能想在初始化的时候也进行这种类型检查。通过设置 `self.first_name `，自动调用 `setter` 方法， 这个方法里面会进行参数的检查，否则就是直接访问`self._first_name` 了。

In [ ]:
Person.first_name.fget

In [ ]:
Person.first_name.fset

In [ ]:
Person.first_name.fdel

## 7. 调用父类方法

In [28]:
# 为了调用父类（超类）的一个方法，可以使用super()函数，比如：
class A:
    def spam(self):
        print('A.spam')
        
class B(A):
    def spam(self):
        print('B.spam')
        super().spam() # call parent spam

In [29]:
b = B()

In [30]:
print(b.spam())

B.spam
A.spam
None


`super()` 函数的一个常见用法是在` __init__() `方法中确保父类被正确的初始化了：

In [31]:
class A:
    def __init__(self):
        self.x = 0
        
class B(A):
    def __init__(self):
        super().__init__()
        self.y = 1

In [32]:
b = B()

In [33]:
b.x

0

In [34]:
b.y

1

`super()` 的另外一个常见用法出现在覆盖`Python`特殊方法的代码中，比如：

In [35]:
class Proxy:
    
    def __init__(self,obj):
        self._obj = obj
        
    # Detegate attribute lookup to internal obj
    def __getattr__(self,name):
        return getattr(self._obj,name)
    
    # Delegate attribute assignment
    def __setattr__(self,name,value):
        if name.startswith('_'):
            super().__setattr__(name,value) # call original.__setattr__
        else:
            setattr(self._obj,name,value)

## 8. 子类中扩展property

In [36]:
class Person:
    
    def __init__(self,name):
        self.name = name 
        
    # Getter function
    @property
    def name(self):
        return self._name
    
    # Setter function
    @name.setter
    def name(self,value):
        if not isinstance(value,str):
            raise TypeError('Expected a string')
        self._name = value
        
    # Deleter function
    @name.deleter
    def name(self):
        raise AttributeError("Can't delete attribute")

下面是一个示例类，它继承自Person并扩展了 name 属性的功能：

In [37]:
class SubPerson(Person):
    @property
    def name(self):
        print('Getting name')
        return super().name
    
    @name.setter
    def name(self,value):
        print('Setting name to',value)
        super(SubPerson,SubPerson).name.__set__(self,value)
        
    @name.deleter
    def name(self):
        print('Deleting nane')
        super(SubPerson,SubPerson).name.__delete__(self)

In [38]:
# 接下来使用这个新类
s = SubPerson('Guido')

Setting name to Guido


In [39]:
s.name

Getting name


'Guido'

In [40]:
s.name = 42

Setting name to 42


TypeError: Expected a string

>> 在子类中扩展一个`property`可能会引起很多不易察觉的问题， 因为一个`property`其实是 `getter`、`setter` 和 `deleter` 方法的集合，而不是单个方法。 因此，当你扩展一个`property`的时候，你需要先确定你是否要重新定义所有的方法还是说只修改其中某一个。

## 9. 创建新的类或实例属性

如果你想创建一个全新的实例属性，可以通过一个描述器类的形式来定义它的功能。下面是一个例子：

In [41]:
class Integer:
    def __init__(self,name):
        self.name = name
        
    def __get__(self,instance,cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]
    def __set__(self,instance,value):
        if not isinstance(value,int):
            raise TypeError('Expected an int')
        instance.__dict__[self.name] = value
        
    def __delete__(self,instance):
        del instance.__dict__[self.name]

一个描述器就是一个实现了三个核心的属性访问操作`(get, set, delete)`的类， 分别为 `__get__() `、`__set__()` 和 `__delete__() `这三个特殊的方法。 这些方法接受一个实例作为输入，之后相应的操作实例底层的字典。

为了使用一个描述器，需将这个描述器的实例作为类属性放到一个类的定义中。例如：

In [42]:
class Point:
    x = Integer('x')
    y = Integer('y')
    
    def __init__(self,x,y):
        self.x = x
        self.y = y

当你这样做后，所有对描述器属性(比如x或y)的访问会被 `__get__() `、`__set__()` 和 `__delete__()` 方法捕获到。例如：

In [43]:
p = Point(2,3)

In [44]:
p.x

2

In [45]:
p.y

3

>> 描述器的一个比较困惑的地方是它只能在类级别被定义，而不能为每个实例单独定义。

最后要指出的一点是，如果你只是想简单的自定义某个类的单个属性访问的话就不用去写描述器了。 这种情况下使用8.6小节介绍的property技术会更加容易。 当程序中有很多重复代码的时候描述器就很有用了 (比如你想在你代码的很多地方使用描述器提供的功能或者将它作为一个函数库特性)。

## 10. 使用延迟计算属性

In [46]:
# 定义一个延迟属性的一种高效方法是通过使用一个描述器类
class lazyproperty:
    
    def __init__(self,func):
        self.func = func
        
    def __get__(self,instance,cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
            setattr(instance,self.func.__name__,value)
            return value

In [47]:
import math

class Circle:
    
    def __init__(self,radius):
        self.radius = radius
        
    @lazyproperty
    def area(self):
        print('Computing area')
        return math.pi * self.radius ** 2
    
    @lazyproperty
    def perimeter(self):
        print('Computing perimeter')
        return 2 * math.pi * self.radius

In [48]:
c = Circle(4.0)

In [49]:
c.radius

4.0

In [50]:
c.area

Computing area


50.26548245743669

In [51]:
c.perimeter

Computing perimeter


25.132741228718345

In [52]:
c.perimeter

25.132741228718345

仔细观察你会发现消息 `Computing area` 和 `Computing perimeter` 仅仅出现一次。

>> 很多时候，构造一个延迟计算属性的主要目的是为了提升性能。 例如，你可以避免计算这些属性值，除非你真的需要它们。 这里演示的方案就是用来实现这样的效果的， 只不过它是通过以非常高效的方式使用描述器的一个精妙特性来达到这种效果的。

## 11. 简化数据结构的初始化

In [53]:
# 可以在一个基类中写一个公用的__init__()函数
import math

class Structurel:
    #Class variable that specifies expected fields
    _fields = []
    
    def __init__(self,*args):
        if len(args) != len(self._fields):
            raise TypeError('Expected {} arguments'.format(len(self._fields))) 
        # Set the arguments
        for name,value in zip(self._fields,args):
            setattr(self,name,value)

In [54]:
# 然后你的类继承这个基类
class Stock(Structurel):
    _fields = ['name','shares','price']
    
class Point(Structurel):
    _fields = ['x','y']
    
class Circle(Structurel):
    _fields = ['radius']
    
    def area(self):
        return math.pi * self.radius ** 2

In [55]:
s = Stock('ACME',50,91.1)

In [56]:
p = Point(2,3)

In [57]:
c = Circle(4.5)

In [58]:
s2 = Stock('ACME',50)

TypeError: Expected 3 arguments

如果还想支持关键字参数，可以将关键字参数设置为实例属性：

In [59]:
class Structure2:
    _fields = []

    def __init__(self, *args, **kwargs):
        if len(args) > len(self._fields):
            raise TypeError('Expected {} arguments'.format(len(self._fields)))

        # Set all of the positional arguments
        for name, value in zip(self._fields, args):
            setattr(self, name, value)

        # Set the remaining keyword arguments
        for name in self._fields[len(args):]:
            setattr(self, name, kwargs.pop(name))

        # Check for any remaining unknown arguments
        if kwargs:
            raise TypeError('Invalid argument(s): {}'.format(','.join(kwargs)))
# Example use
if __name__ == '__main__':
    class Stock(Structure2):
        _fields = ['name', 'shares', 'price']

    s1 = Stock('ACME', 50, 91.1)
    s2 = Stock('ACME', 50, price=91.1)
    s3 = Stock('ACME', shares=50, price=91.1)
    # s3 = Stock('ACME', shares=50, price=91.1, aa=1)

In [60]:
# 你还能将不在 _fields 中的名称加入到属性中去：
class Structure3:
    # Class variable that specifies expected fields
    _fields = []

    def __init__(self, *args, **kwargs):
        if len(args) != len(self._fields):
            raise TypeError('Expected {} arguments'.format(len(self._fields)))

        # Set the arguments
        for name, value in zip(self._fields, args):
            setattr(self, name, value)

        # Set the additional arguments (if any)
        extra_args = kwargs.keys() - self._fields
        for name in extra_args:
            setattr(self, name, kwargs.pop(name))

        if kwargs:
            raise TypeError('Duplicate values for {}'.format(','.join(kwargs)))

# Example use
if __name__ == '__main__':
    class Stock(Structure3):
        _fields = ['name', 'shares', 'price']

    s1 = Stock('ACME', 50, 91.1)
    s2 = Stock('ACME', 50, 91.1, date='8/2/2012')

>> 当你需要使用大量很小的数据结构类的时候， 相比手工一个个定义` __init__()` 方法而已，使用这种方式可以大大简化代码。

## 12. 定义接口或者抽象基类

In [61]:
# 使用 abc 模块可以很轻松的定义抽象基类：
from abc import ABCMeta,abstractmethod

class IStream(metaclass=ABCMeta):
    @abstractmethod
    def read(self,maxbytes=-1):
        pass
    
    @abstractmethod
    def write(self,data):
        pass

抽象类的一个特点是它不能直接被实例化，比如你想像下面这样做是不行的：

In [62]:
a = IStream()
a

TypeError: Can't instantiate abstract class IStream with abstract methods read, write

抽象类的**目的**就是**让别的类继承它并实现特定的抽象方法**：

In [63]:
class SocketStream(IStream):
    def read(self, maxbytes=-1):
        pass

    def write(self, data):
        pass

抽象基类的一个主要用途是在代码中检查某些类是否为特定类型，实现了特定接口：

In [64]:
def serialize(obj, stream):
    if not isinstance(stream, IStream):
        raise TypeError('Expected an IStream')
    pass

除了继承这种方式外，还可以通过注册方式来让某个类实现抽象基类：

In [65]:
import io

# Register the built-in I/O classes as supporting our interface
IStream.register(io.IOBase)

# Open a normal file and type check
f = open('foo.txt')
isinstance(f, IStream) # Returns True

True

**@abstractmethod** 还能注解静态方法、类方法和 `properties` 。 你只需保证这个注解紧靠在函数定义前即可

In [66]:
class A(metaclass=ABCMeta):
    @property
    @abstractmethod
    def name(self):
        pass
    
    @name.setter
    @abstractmethod
    def name(self,value):
        pass
    
    @classmethod
    @abstractmethod
    def method1(cls):
        pass
    
    @staticmethod
    @abstractmethod
    def method2():
        pass

>> 标准库中有很多用到抽象基类的地方。`collections` 模块定义了很多跟容器和迭代器(序列、映射、集合等)有关的抽象基类。` numbers `库定义了跟数字对象(整数、浮点数、有理数等)有关的基类。`io` 库定义了很多跟`I/O`操作相关的基类。

##  实现数据模型的类型约束

在这个问题，你需要在对某些实例属性赋值时进行检查。所以你要自定义属性赋值函数，这种情况下最好使用描述器。

In [67]:
# Base class. Uses a descriptor to set a value
class Descriptor:
    def __init__(self,name=None,**opts):
        self.name = name
        for key,value in opts.items():
            setattr(self,key,value)
    def __set__(self,instance,value):
        instance.__dict__[self.name] = value
        
# Descriptor for enforcing types
class Typed(Descriptor):
    expected_type = type(None)
    
    def __set__(self,instance,value):
        if not isinstance(value,self.expected_type):
            raise TypeError('expected '+ str(self.expected_type))
            super().__set__(instance,value)
            
# Descriptor for enforcing values
class Unsigned(Descriptor):
    def __set__(self,instance,value):
        if value < 0:
            raise ValueError('Expected >= 0')
        super().__set__(instance,value)

class MaxSized(Descriptor):
    def __init__(self,name=None,**opts):
        if 'size' not in opts:
            raise TypeError('missing size option')
            
    def __set__(self,instance,value):
        if len(value) >= self.size:
            raise ValueError('size must be <' + str(self.size))
        super().__set__(instance,value)

In [68]:
# 下面是我们实际定义的各种不同的数据类型
class Integer(Typed):
    expected_type = int
    
class UnsignedInteger(Integer,Unsigned):
    pass

class Float(Typed):
    expected_type = float
    
class UnsignedFloat(Float,Unsigned):
    pass

class String(Typed):
    expected_type = str
    
class SizedString(String,MaxSized):
    pass

In [69]:
# 然后使用这些自定义数据类型，定义一个类
class Stock:
    # Specify constraints
    name = SizedString('name',size=8)
    shares = UnsignedInteger('shares')
    price = UnsignedFloat('price')
    
    def __init__(self,name,shares,price):
        self.name = name
        self.shares = shares
        self.price = price

In [70]:
# 还有一些技术可以简化上面的代码，其中一种是使用类装饰器：
# Class decorator to apply constraints
def check_attributes(**kwargs):
    def decorate(cls):
        for key,value in kwargs.items():
            if isinstance(value,Descriptor):
                value.name = value
                setattr(cls,key,value)
            else:
                setattr(cls,key,value(key))
        return cls
    return decorate

# Example
@check_attributes(name=SizedString(size=8),
                 shares=UnsignedInteger,
                 price=UnsignedFloat)

class Stock:
    def __init__(self,name,shares,price):
        self.name = name
        self.shares = shares
        self.price = price

In [71]:
# 另外一种是使用元类
# A metaclass that applies checking
class checkmeta(type):
    def __new__(cls,clsname,bases,methods):
        # Attach attribute names to the descriptors
        for key,value in methods.items():
            if isinstance(value,Descriptor):
                value.name = key
            return type.__new__(cls,clsname,bases,methods)
        
# Example
class Stock2(metaclass=checkmeta):
    name = SizedString(size=8)
    shares = UnsignedInteger()
    price = UnsignedFloat()
    
    def __init__(self,name,shares,price):
        self.name = name
        self.shares = shares
        self.price = price

首先，在 `Descriptor `基类中你会看到有个` __set__() `方法，却没有相应的 `__get__()` 方法。 如果一个描述仅仅是从底层实例字典中获取某个属性值的话，那么没必要去定义 `__get__()` 方法。

所有描述器类都是基于混入类来实现的。比如 `Unsigned `和 `MaxSized` 要跟其他继承自 `Typed` 类混入。 这里利用多继承来实现相应的功能。

混入类的一个比较难理解的地方是，调用` super() `函数时，你并不知道究竟要调用哪个具体类。 你需要跟其他类结合后才能正确的使用，也就是必须合作才能产生效果。



## 实现自定义容器

    collections定义了很多抽象的基类，当你想自定义容器类时他们会非常有用。当你想让你的类支持迭代的时候，那么让你的类继承collections.Iterable即可

In [72]:
import collections

class A(collections.Iterable):
    pass

    不过你需要实现collections.Iterable所有的抽象方法，否则会报错的：

In [73]:
a = A()

TypeError: Can't instantiate abstract class A with abstract methods __iter__

    你只要实现`__iter__()`方法就不会报错了，你可以试着去实例化一个对象，在错误提示中找到实现哪些方法：


In [74]:
import collections

In [75]:
collections.Sequence()

/Users/lidianxiang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  """Entry point for launching an IPython kernel.


TypeError: Can't instantiate abstract class Sequence with abstract methods __getitem__, __len__

    下面是一个简单的实例，继承自上面Sequence抽象类，并且实现元素按照顺序存储：

In [76]:
import bisect

class SortedItems(collections.Sequence):
    def __init__(self,initial=None):
        self._items = sorted(initial) if initial is not None else []
        
    # Required sequence mothods
    def __getitem__(self,index):
        return self._items[index]
    
    def __len__(self):
        return len(self._items)
    
    # Method for adding an item in the right location
    def add(self,item):
        bisect.insort(self._items,item)

In [77]:
items = SortedItems([5,1,3])

In [78]:
print(list(items))

[1, 3, 5]


In [79]:
print(items[0],items[-1])

1 5


In [80]:
items.add(2)

In [81]:
print(list(items))

[1, 2, 3, 5]


      可以看到，SortedItems跟普通的序列没什么两样，支持所有常用操作，包括索引、迭代、包含判断，甚至是切片操作。

>> 这里面使用到了` bisect` 模块，它是一个在排序列表中插入元素的高效方式。可以保证元素插入后还保持顺序。

## 属性的代理访问


简单来说，代理是一种编程模式，它将某个操作转移给另外一个对象来实现。 最简单的形式可能是像下面这样：

In [82]:
class A:
    def spam(self,x):
        pass
    
    def foo(self):
        pass
    
class B1:
    '''简单的代理'''
    def __init__(self):
        self._a = A()
        
    def spam(self,x):
        # Delegate to the internal self._a instance
        return self._a.spam(x)
    
    def foo(self):
        # Delegate to the internal self._a instance
        return self._a.foo()
    
    def bar(self):
        pass

如果仅仅就两个方法需要代理，那么像这样写就足够了。但是，如果有大量的方法需要代理， 那么使用 __getattr__() 方法或许或更好些：

In [83]:
class B2:
    """使用__getattr__的代理，代理方法比较多时候"""
    
    def __init__(self):
        self._a = A()
        
    def bar(self):
        pass
    
    # Expose all the methods defined on class A
    def __getattr__(self,name):
        """这个方法在访问的attribute不存在的时候被调用"""
        return getattr(self._a,name)

`__getattr__` 方法是在访问attribute不存在的时候被调用，使用演示

In [84]:
# 另外一个代理例子是实现代理模式：
class Proxy:
    def __init__(self,obj):
        self._obj = obj
        
    def __getattr__(self,name):
        print('getattr:',name)
        return getattr(self._obj,name)
    
    def __setattr__(self,name,value):
        if name.startswith('_'):
            super().__setattr__(name,value)
        else:
            print('setattr:',name,value)
            setattr(self._obj,name,value)
            
    def __delattr__(self,name):
        if name.startswith('_'):
            super().__delattr__(name)
        else:
            print('delattr:',name)
            delattr(self._obj,name)

In [85]:
# 使用这个代理时，你只需要用它来包装下其它类即可：
class Spam:
    def __init__(self,x):
        self.x = x
        
    def bar(self,y):
        print('Spam.bar:',self.x,y)

In [86]:
s = Spam(2)

In [87]:
p = Proxy(5)

In [88]:
print(p.x)

getattr: x


AttributeError: 'int' object has no attribute 'x'

## 在类中定义多个构造器

In [89]:
# 为了实现多个构造器，你需要使用到类方法。
import time 

class Date:
    """方法一：使用类方法"""
    def __init__(self,year,month,day):
        self.year = year
        self.month = month
        self.day = day
        
    # Alternate construtor
    @classmethod
    def today(cls):
        t = time.localtime()
        return cls(t.tm_year,t.tm_mon,t.tm_mday)

In [90]:
# 直接调用类方法即可
a = Date(2012,12,21)

In [91]:
b = Date.today()
print(b)

类方法的一个主要用途就是定义多个构造器。它接受一个 `class `作为第一个参数`(cls)`。 你应该注意到了这个类被用来创建并返回最终的实例。在继承时也能工作的很好：

In [92]:
class NewDate(Date):
    pass

c = Date.today() # Creates an instance of Date (cls=Date)
d = NewDate.today() # Creates an instance of NewDate (cls=NewDate)

## 创建不调用init方法的实例


In [93]:
# 可以通过__new__（）方法创建一个未初始化的实例。
class Date:
    def __init__(self,year,month,day):
        self.year = year
        self.month = month
        self.day = day

In [94]:
d = Date.__new__(Date)

In [95]:
d

In [96]:
d.year

AttributeError: 'Date' object has no attribute 'year'

结果可以看到，这个Date实例的属性year还不存在，所以你需要手动初始化：

In [97]:
date = {'year':2012,'month':8,'day':29}

In [98]:
for key,value in date.items():
    setattr(d,key,value)

In [99]:
d.year

2012

In [100]:
d.month

8

In [101]:
d.day

29

当我们在反序列对象或者实现某个类方法构造函数时需要绕过` __init__() `方法来创建对象。 例如，对于上面的Date来讲，有时候你可能会像下面这样定义一个新的构造函数 `today()` ：



In [102]:
from time import localtime

class Date:
    def __init__(self,year,month,day):
        self.year = year
        self.month = month
        self.day = day
        
    @staticmethod
    def today(cls):
        d = cls.__new__(cls)
        t = localtime()
        d.year = t.tm_year
        d.month = t.tm_mon
        d.day = t.tm_mday
        return d

## 利用Mixins扩展类功能

In [103]:
class LoggedMappingMixin:
    """
    Add logging to get/set/delete operations for debugging.
    """
    __slots__ = ()  # 混入类都没有实例变量，因为直接实例化混入类没有任何意义

    def __getitem__(self, key):
        print('Getting ' + str(key))
        return super().__getitem__(key)

    def __setitem__(self, key, value):
        print('Setting {} = {!r}'.format(key, value))
        return super().__setitem__(key, value)

    def __delitem__(self, key):
        print('Deleting ' + str(key))
        return super().__delitem__(key)


class SetOnceMappingMixin:
    '''
    Only allow a key to be set once.
    '''
    __slots__ = ()

    def __setitem__(self, key, value):
        if key in self:
            raise KeyError(str(key) + ' already set')
        return super().__setitem__(key, value)


class StringKeysMappingMixin:
    '''
    Restrict keys to strings only
    '''
    __slots__ = ()

    def __setitem__(self, key, value):
        if not isinstance(key, str):
            raise TypeError('keys must be strings')
        return super().__setitem__(key, value)


这些类单独使用起来没有任何意义，事实上如果你去实例化任何一个类，除了产生异常外没任何作用。 它们是用来通过多继承来和其他映射对象混入使用的。例如：

In [104]:
class LoggedDict(LoggedMappingMixin, dict):
    pass

d = LoggedDict()
d['x'] = 23
print(d['x'])
del d['x']

from collections import defaultdict

class SetOnceDefaultDict(SetOnceMappingMixin, defaultdict):
    pass


d = SetOnceDefaultDict(list)
d['x'].append(2)
d['x'].append(3)

Setting x = 23
Getting x
23
Deleting x


In [105]:
d['x'] = 23

KeyError: 'x already set'

## 实现状态对象或者状态机

在很多程序中，有些对象会根据状态的不同来执行不同的操作。比如考虑如下的一个连接对象

In [106]:
class Connection:
    """普通方案，好多个判断语句，效率低下"""
    def __init__(self):
        self.state = 'CLOSED'
        
    def read(self):
        if self.state != 'OPEN':
            raise RuntimeError('Not open')
        print('reading')
    
    def write(self,data):
        if self.state != "OPEN":
            raise RuntimeError('Not open')
        print('writing')
        
    def open(self):
        if self.state == "OPEN":
            raise RuntimeError("Already open")
        self.state = 'OPEN'
    def close(self):
        if self.state == 'CLOSED':
            raise RuntimeError('Already closed')
        self.state = 'CLOSED'

>> 这样写有很多缺点，首先是代码太复杂了，好多的条件判断。其次是执行效率变低， 因为一些常见的操作比如read()、write()每次执行前都需要执行检查。

In [107]:
# 一个更好的办法是为每一个状态定义一个对象：
class Connection1:
    """新方案——对每个状态定义一个类"""

    def __init__(self):
        self.new_state(ClosedConnectionState)

    def new_state(self, newstate):
        self._state = newstate
        # Delegate to the state class

    def read(self):
        return self._state.read(self)

    def write(self, data):
        return self._state.write(self, data)

    def open(self):
        return self._state.open(self)

    def close(self):
        return self._state.close(self)

# Connection state base class
class ConnectionState:
    @staticmethod
    def read(conn):
        raise NotImplementedError
        
    @staticmethod
    def write(conn,data):
        raise NotImplementedError
        
    @staticmethod
    def open(conn):
        raise NotImplementedError
        
    @staticmethod
    def close(conn):
        raise NotImplementedError
        
# Implementation of differentt states
class ClosedConnectionState(ConnectionState):
    @staticmethod
    def read(conn):
        raise RuntimeError('Not open')
    
    @staticmethod
    def write(conn,data):
        raise RuntimeError('Not open')
    
    @staticmethod
    def open(conn):
        conn.new_state(OpenConnectionState)

    @staticmethod
    def close(conn):
        raise RuntimeError('Already closed')
        
class OpenConnectionState(ConnectionState):
    @staticmethod
    def read(conn):
        print('reading')
        
    @staticmethod
    def write(conn,data):
        print('writing')
    
    @staticmethod
    def open(conn):
        raise RuntimeError('Already open')
        
    @staticmethod
    def close(conn):
        conn.new_state(ClosedConnectionState)

In [108]:
c = Connection1()

In [109]:
c._state

__main__.ClosedConnectionState

In [110]:
c.read()

RuntimeError: Not open

In [ ]:
c.open()

In [ ]:
c._state

In [ ]:
c.read()

In [ ]:
c.write('hello,world')

In [111]:
c.close()

RuntimeError: Already closed

In [ ]:
c._state

>> 如果代码中出现太多的条件判断语句的话，代码就会变得难以维护和阅读。 这里的解决方案是将每个状态抽取出来定义成一个类。

## 通过字符串调用对象方法

In [112]:
# 最简单的办法是使用getattr()
import math

class Point:
    def __init__(self,x,y):
        self.x = x 
        self.y = y
        
    def __repr__(self):
        return 'Point({!r:},{!r:})'.format(self.x,self.y)
    
    def distance(self,x,y): 
        return math.hypot(self.x -x ,self.y -y)

In [113]:
p = Point(2,3)

In [114]:
d = getattr(p,'distance')(0,0) # Calls p.distance(0,0) 

In [115]:
d

3.605551275463989

## 实现访问者模式

这里遇到的问题在编程领域中是很普遍的，有时候会构建一个由大量不同对象组成的数据结构。 假设你要写一个表示数学表达式的程序，那么你可能需要定义如下的类：

In [116]:
class Node:
    pass

class UnaryOperator(Node):
    def __init__(self, operand):
        self.operand = operand

class BinaryOperator(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOperator):
    pass

class Sub(BinaryOperator):
    pass

class Mul(BinaryOperator):
    pass

class Div(BinaryOperator):
    pass

class Negate(UnaryOperator):
    pass

class Number(Node):
    def __init__(self, value):
        self.value = value

In [117]:
# Representation of 1 + 2 * (3 - 4) / 5
t1 = Sub(Number(3), Number(4))
t2 = Mul(Number(2), t1)
t3 = Div(t2, Number(5))
t4 = Add(Number(1), t3)

这样做的问题是对于每个表达式，每次都要重新定义一遍，有没有一种更通用的方式让它支持所有的数字和操作符呢。 这里我们使用访问者模式可以达到这样的目的：

In [118]:
class NodeVisitor:
    def visit(self, node):
        methname = 'visit_' + type(node).__name__
        meth = getattr(self, methname, None)
        if meth is None:
            meth = self.generic_visit
        return meth(node)

    def generic_visit(self, node):
        raise RuntimeError('No {} method'.format('visit_' + type(node).__name__))

为了使用这个类，可以定义一个类继承它并且实现各种 visit_Name() 方法，其中Name是node类型。 例如，如果你想求表达式的值，可以这样写：

In [119]:
class Evaluator(NodeVisitor):
    def visit_Number(self, node):
        return node.value

    def visit_Add(self, node):
        return self.visit(node.left) + self.visit(node.right)

    def visit_Sub(self, node):
        return self.visit(node.left) - self.visit(node.right)

    def visit_Mul(self, node):
        return self.visit(node.left) * self.visit(node.right)

    def visit_Div(self, node):
        return self.visit(node.left) / self.visit(node.right)

    def visit_Negate(self, node):
        return -node.operand


In [120]:
 e = Evaluator()

In [121]:
e.visit(t4)

0.6

## 不用递归实现访问者模式

通过巧妙的使用生成器可以在树遍历或搜索算法中消除递归。 在8.21小节中，我们给出了一个访问者类。 下面我们利用一个栈和生成器重新实现这个类

In [122]:
import types

class Node:
    pass

class NodeVisitor:
    def visit(self, node):
        stack = [node]
        last_result = None
        while stack:
            try:
                last = stack[-1]
                if isinstance(last, types.GeneratorType):
                    stack.append(last.send(last_result))
                    last_result = None
                elif isinstance(last, Node):
                    stack.append(self._visit(stack.pop()))
                else:
                    last_result = stack.pop()
            except StopIteration:
                stack.pop()

        return last_result

    def _visit(self, node):
        methname = 'visit_' + type(node).__name__
        meth = getattr(self, methname, None)
        if meth is None:
            meth = self.generic_visit
        return meth(node)

    def generic_visit(self, node):
        raise RuntimeError('No {} method'.format('visit_' + type(node).__name__))

如果你使用这个类，也能达到相同的效果。事实上你完全可以将它作为上一节中的访问者模式的替代实现。 考虑如下代码，遍历一个表达式的树：



In [123]:
class UnaryOperator(Node):
    def __init__(self, operand):
        self.operand = operand

class BinaryOperator(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOperator):
    pass

class Sub(BinaryOperator):
    pass

class Mul(BinaryOperator):
    pass

class Div(BinaryOperator):
    pass

class Negate(UnaryOperator):
    pass

class Number(Node):
    def __init__(self, value):
        self.value = value

# A sample visitor class that evaluates expressions
class Evaluator(NodeVisitor):
    def visit_Number(self, node):
        return node.value

    def visit_Add(self, node):
        return self.visit(node.left) + self.visit(node.right)

    def visit_Sub(self, node):
        return self.visit(node.left) - self.visit(node.right)

    def visit_Mul(self, node):
        return self.visit(node.left) * self.visit(node.right)

    def visit_Div(self, node):
        return self.visit(node.left) / self.visit(node.right)

    def visit_Negate(self, node):
        return -self.visit(node.operand)

if __name__ == '__main__':
    # 1 + 2*(3-4) / 5
    t1 = Sub(Number(3), Number(4))
    t2 = Mul(Number(2), t1)
    t3 = Div(t2, Number(5))
    t4 = Add(Number(1), t3)
    # Evaluate it
    e = Evaluator()
    print(e.visit(t4))  # Outputs 0.6

0.6


## 循环引用数据结构的内存管理

一个简单的循环引用数据结构例子就是一个树形结构，双亲节点有指针指向孩子节点，孩子节点又返回来指向双亲节点。 这种情况下，可以考虑使用 weakref 库中的弱引用。例如：

In [124]:
import weakref

class Node:
    def __init__(self, value):
        self.value = value
        self._parent = None
        self.children = []

    def __repr__(self):
        return 'Node({!r:})'.format(self.value)

    # property that manages the parent as a weak-reference
    @property
    def parent(self):
        return None if self._parent is None else self._parent()

    @parent.setter
    def parent(self, node):
        self._parent = weakref.ref(node)

    def add_child(self, child):
        self.children.append(child)
        child.parent = self

In [125]:
root = Node('parent')

In [126]:
c1 = Node('child')

In [127]:
root.add_child(c1)

In [128]:
print(c1.parent)

Node('parent')


In [129]:
del root

In [130]:
print(c1.parent)

None


循环引用的数据结构在`Python`中是一个很棘手的问题，因为正常的垃圾回收机制不能适用于这种情形。 

In [131]:
# Class just to illustrate when deletion occurs
class Data:
    def __del__(self):
        print('Data.__del__')

# Node class involving a cycle
class Node:
    def __init__(self):
        self.data = Data()
        self.parent = None
        self.children = []

    def add_child(self, child):
        self.children.append(child)
        child.parent = self


In [132]:
a = Data() 

In [133]:
del a

Data.__del__


In [134]:
a = Node() 

In [135]:
del a

Data.__del__


In [136]:
a = Node() 

In [137]:
a.add_child(Node()) 

In [138]:
del a

可以看到，最后一个的删除时打印语句没有出现。原因是Python的垃圾回收机制是基于简单的引用计数。 当一个对象的引用数变成0的时候才会立即删除掉。而对于循环引用这个条件永远不会成立。 因此，在上面例子中最后部分，父节点和孩子节点互相拥有对方的引用，导致每个对象的引用计数都不可能变成0。



## 让类支持比较操作

Python类对每个比较操作都需要实现一个特殊方法来支持。 例如为了支持>=操作符，你需要定义一个 `__ge__()` 方法。 尽管定义一个方法没什么问题，但如果要你实现所有可能的比较方法那就有点烦人了。

装饰器 `functools.total_ordering `就是用来简化这个处理的。 使用它来装饰一个来，你只需定义一个` __eq__() `方法， 外加其他方法`(__lt__, __le__, __gt__, or __ge__)`中的一个即可。 然后装饰器会自动为你填充其它比较方法。

作为例子，我们构建一些房子，然后给它们增加一些房间，最后通过房子大小来比较它们：

In [139]:
from functools import total_ordering

class Room:
    def __init__(self, name, length, width):
        self.name = name
        self.length = length
        self.width = width
        self.square_feet = self.length * self.width

@total_ordering
class House:
    def __init__(self, name, style):
        self.name = name
        self.style = style
        self.rooms = list()

    @property
    def living_space_footage(self):
        return sum(r.square_feet for r in self.rooms)

    def add_room(self, room):
        self.rooms.append(room)

    def __str__(self):
        return '{}: {} square foot {}'.format(self.name,
                self.living_space_footage,
                self.style)

    def __eq__(self, other):
        return self.living_space_footage == other.living_space_footage

    def __lt__(self, other):
        return self.living_space_footage < other.living_space_footage

Data.__del__
Data.__del__


In [140]:
# Build a few houses, and add rooms to them
h1 = House('h1', 'Cape')
h1.add_room(Room('Master Bedroom', 14, 21))
h1.add_room(Room('Living Room', 18, 20))
h1.add_room(Room('Kitchen', 12, 16))
h1.add_room(Room('Office', 12, 12))
h2 = House('h2', 'Ranch')
h2.add_room(Room('Master Bedroom', 14, 21))
h2.add_room(Room('Living Room', 18, 20))
h2.add_room(Room('Kitchen', 12, 16))
h3 = House('h3', 'Split')
h3.add_room(Room('Master Bedroom', 14, 21))
h3.add_room(Room('Living Room', 18, 20))
h3.add_room(Room('Office', 12, 16))
h3.add_room(Room('Kitchen', 15, 17))
houses = [h1, h2, h3]
print('Is h1 bigger than h2?', h1 > h2) # prints True
print('Is h2 smaller than h3?', h2 < h3) # prints True
print('Is h2 greater than or equal to h1?', h2 >= h1) # Prints False
print('Which one is biggest?', max(houses)) # Prints 'h3: 1101-square-foot Split'
print('Which is smallest?', min(houses)) # Prints 'h2: 846-square-foot Ranch'

Is h1 bigger than h2? True
Is h2 smaller than h3? True
Is h2 greater than or equal to h1? False
Which one is biggest? h3: 1101 square foot Split
Which is smallest? h2: 846 square foot Ranch


## 创建缓存实例

这种通常是因为你希望相同参数创建的对象时单例的。 在很多库中都有实际的例子，比如 logging 模块，使用相同的名称创建的 logger 实例永远只有一个。例如：

In [141]:
import logging

In [142]:
a = logging.getLogger('foo') 

In [143]:
b = logging.getLogger('bar') 

In [144]:
a is b

False

In [145]:
c = logging.getLogger('foo') 

In [146]:
a is c

True

为了达到这样的效果，你需要使用一个和类本身分开的工厂函数，例如：

In [147]:
# The class in question
class Spam:
    def __init__(self, name):
        self.name = name

# Caching support
import weakref
_spam_cache = weakref.WeakValueDictionary()
def get_spam(name):
    if name not in _spam_cache:
        s = Spam(name)
        _spam_cache[name] = s
    else:
        s = _spam_cache[name]
    return s

In [148]:
a = get_spam('foo') 

In [149]:
b = get_spam('bar') 

In [150]:
a is b

False

In [151]:
c = get_spam('foo') 

In [152]:
a is c

True

首先是这里使用到了一个全局变量，并且工厂函数跟类放在一块。我们可以通过将缓存代码放到一个单独的缓存管理器中：



In [153]:
import weakref

class CachedSpamManager:
    def __init__(self):
        self._cache = weakref.WeakValueDictionary()

    def get_spam(self, name):
        if name not in self._cache:
            s = Spam(name)
            self._cache[name] = s
        else:
            s = self._cache[name]
        return s

    def clear(self):
            self._cache.clear()

class Spam:
    manager = CachedSpamManager()
    def __init__(self, name):
        self.name = name

    def get_spam(name):
        return Spam.manager.get_spam(name)

这样的话代码更清晰，并且也更灵活，我们可以增加更多的缓存管理机制，只需要替代manager即可。

还有一点就是，我们暴露了类的实例化给用户，用户很容易去直接实例化这个类，而不是使用工厂方法，如：



In [154]:
a = Spam('foo') 

In [155]:
b = Spam('foo') 

In [156]:
a is b

False

有几种方式可以防止用户这样做，第一个是将类的名字修改为以下划线`(_)`开头，提示用户别直接调用它。 第二种就是让这个类的 `__init__() `方法抛出一个异常，让它不能被初始化：

In [157]:
class Spam:
    def __init__(self, *args, **kwargs):
        raise RuntimeError("Can't instantiate directly")

    # Alternate constructor
    @classmethod
    def _new(cls, name):
        self = cls.__new__(cls)
        self.name = name



然后修改缓存管理器代码，使用 Spam._new() 来创建实例，而不是直接调用 Spam() 构造函数：

In [158]:
class CachedSpamManager2:
    def __init__(self):
        self._cache = weakref.WeakValueDictionary()

    def get_spam(self, name):
        if name not in self._cache:
            temp = Spam3._new(name)  # Modified creation
            self._cache[name] = temp
        else:
            temp = self._cache[name]
        return temp

    def clear(self):
            self._cache.clear()

class Spam3:
    def __init__(self, *args, **kwargs):
        raise RuntimeError("Can't instantiate directly")

    # Alternate constructor
    @classmethod
    def _new(cls, name):
        self = cls.__new__(cls)
        self.name = name
        return self

最后这样的方案就已经足够好了。 缓存和其他构造模式还可以使用9.13小节中的元类实现的更优雅一点(使用了更高级的技术)。